In [3]:
import pandas as pd
import numpy as np
from xlsxwriter.utility import xl_rowcol_to_cell

# ----------------------------------------------FILL IN THE INFORMATION HERE-----------------------------------------------------
# start  is the beginning of the auditing period and end is the end of the monitoring period. Date format is (year,month,day).
start = pd.Timestamp(2020,1,1)
end = pd.Timestamp(2021,7,30)
# report_name will be the name of the excel file generated be sure to remember to include .xslx at the end
report_name = "DUI-Test_Court_Audit.xlsx"
# ----------------------------------------------DONT CHANGE ANYTHING BEYOND THIS POINT-----------------------------------------------------


# ------------------------------------------CASE LIST------------------------------------------------------------------------------------------

cl_data = pd.read_csv('caselist.csv', parse_dates = ["Admission Date"])

# clean data to needed data types
cl_data['Admission Date'] = pd.to_datetime(cl_data['Admission Date'])

# cast all blanks to "-1", format discharge date, reason for active 
cl_data["Age Began Using Drugs"] = cl_data["Age Began Using Drugs"].replace([0],-1)
cl_data["Age Began Using Alcohol"] = cl_data["Age Began Using Alcohol"].replace([0],-1)
cl_data["Discharge Date"] = cl_data["Discharge Date"].replace([np.nan], "NA")
cl_data = cl_data.fillna(-1)
cl_data['Discharge Reason']=cl_data['Discharge Reason'].mask(cl_data['Discharge Reason'].eq(-1),cl_data['Discharge Date'])
cl_data['Drug Court Case Outcome']=cl_data['Drug Court Case Outcome'].mask(cl_data['Drug Court Case Outcome'].eq(-1),cl_data['Discharge Date'])
cl_data['Discharge Housing Status']=cl_data['Discharge Housing Status'].mask(cl_data['Discharge Housing Status'].eq(-1),cl_data['Discharge Date'])
cl_data['Employment Status at Discharge']=cl_data['Employment Status at Discharge'].mask(cl_data['Employment Status at Discharge'].eq(-1),cl_data['Discharge Date'])
cl_data['Education Level at Discharge']=cl_data['Education Level at Discharge'].mask(cl_data['Education Level at Discharge'].eq(-1),cl_data['Discharge Date'])
cl_data['Improvement in Education Level']=cl_data['Improvement in Education Level'].mask(cl_data['Improvement in Education Level'].eq(-1),cl_data['Discharge Date'])
cl_data['Improvement in Employment Status']=cl_data['Improvement in Employment Status'].mask(cl_data['Improvement in Employment Status'].eq(-1),cl_data['Discharge Date'])
cl_data['Sentence/Disposition at Discharge']=cl_data['Sentence/Disposition at Discharge'].mask(cl_data['Sentence/Disposition at Discharge'].eq(-1),cl_data['Discharge Date'])


#obscure SID, FDOC, SSN
cl_data.loc[cl_data['SID'] != -1, 'SID'] = "Redacted"
cl_data.loc[cl_data['FDOC'] != -1, 'FDOC'] = "Redacted"
cl_data.loc[cl_data['SSN'] != -1, 'SSN'] = "Redacted"


# delete uneeded rows  
cl_data.drop(["Age at Screening", "Current Age", "Days From Acceptance to SA Treatment", "Sentence/Disposition at Screening","Primary Offense Charge Code","Days From Arrest to SA Treatment", "Days In Jail","Days In Jail By Bench Warrant",
"Days In Jail By Drug Court Sanction", "Days In Jail By New Offense", "Days In Jail By Pre-Admission", "Days of 12 Step Program",  "Days Rearrested in Program",
"Drug Court Control Group","First Treatment Date", "History of IV Use","In SA Treatment Program At Screening","IV Drug User", "Marital Status at Screening",
"Medication Compliance", "Medical Insurance Status", "Number of 12 Step Program Meetings Attended","Number of Bench Warrants", "Number of Days in Drug Court",
"Days Rearrested Post Program", "Number of Days in Phase I", "Number of Days in Phase II", "Number of Days in Phase III", "Number of Days in Phase IV", "Number of Days in Phase IV",
"Number of Felonies", "CaseStatus", "Number of Incentives", "Number of Monitoring with Contact Made - No Violation", "Number of Overall Positive Drug Tests", "Number of Other Offense",
"Number of Misdemeanors", "Number of Sanctions", "Number of Status Offenses", "Number of Scheduled Drug Court Reviews", "Percent of Positive Drug Tests", "Positive Drug Test In Program",
"1st Positive Drug Test (In Program)", "Reoffend Status Offense In Program", "Reoffend Status Offense Post Program", "Retention Days", "Deceased Date", "Number Of Ancillary Sessions",
"Number of Sessions", "SA Early Intervention/Education", "SA Intensive Outpatient", "SA Outpatient", "SA Outpatient Detox", "SA Residential", "SA Sub Acute Detox",
"SA Treatment Contact Hours", "Qualifying Sentencing Score", "Rearrest Date in Program", "Rearrest Date Post Program", "Rearrest in Program", "Case Management/Support Coordination Days",
"Co-occurring Treatment Service Days", "MH Treatment Contact Hours", "MH - Doctor/Medication Review Hours", "MH - Assertive Community Treatment (ACT) Hours",
"MH - Case Management/Support Coordination Hours", "MH - Inpatient Hospitalization/Partial Day Hospitalization Hours", "MH - Co-occurring Treatment Services Hours",
 "MH - Therapy Services Hours", "ClientID",  "PersonGUID", "MH - Residential Hours", "MH - Crisis Residential/Intensive Crisis Stabilization Hours",
 "MH - Employment Services Hours", "MH - Community Based Services Hours",  "Total Number of Drug Tests", "Days From Referral To Screening","Days From Screening To Acceptance", "PlacedInFosterHome",
 "Sobriety Days", "Drug of Choice-Secondary", "Drug of Choice-Tertiary", "Ethnicity", "Improvement in Mental Health", "Improvement in Quality of Life", "Prior Substance Abuse",
"Substance Abuse Disorder at Screening","Zip","Recommended Treatment Modality At Screening",  "Convicted Offense Category in Program", "FullName", "CourtID","ADDRESS","CaseID", "Prior Subsance Abuse Treatment"],axis=1,inplace= True)

# choose date range for cohort
cl_data =cl_data[(cl_data['Admission Date']>= start) & (cl_data['Admission Date']<= end)]  

# find rows with blank and remove deucplicat rows
cl_report = cl_data[cl_data.values == -1]
cl_report.drop_duplicates(inplace=True)


# ------------------------------------------SCREENING REPORT- CASE INFORMATION----------------------------------------------------------------------------------------
ci_data = ci_data = pd.read_csv('p_ScreeningExport_GetCaseInformation.csv', parse_dates = ["ScreeningDate"])

# clean data to needed data types
ci_data['ScreeningDate'] = pd.to_datetime(ci_data['ScreeningDate'])

# fill in blank rows with -1
ci_data = ci_data.fillna(-1)

# delete uneeded rows  
ci_data.drop(["ScreeningID","LastName","FirstName", "MiddleName","DOB",  "DrugCourtApproach","PrimaryOffense", "Counts", "OffenseCategory", "OffenseType","OffenseDate", "IncidentOffense",
"ArrestDate", "FirstAppearanceDate", "ArraignmentDate", "PreliminaryHearingDate", "PretrialHearingDate","CustodyStatus", "BloodAlcoholContent", "IsWeaponUsed", "AdjudicationDate", "ProbationExpirationDate", "Sentence",
"JailStatus", "JailAdmitDate", "JailReleasedDate"],axis=1,inplace= True)

# choose date range for cohort
ci_data =ci_data[(ci_data['ScreeningDate']>= start) & (ci_data['ScreeningDate']<= end)]  

# find rows with blank and remove deucplicat rows
ci_report = ci_data[ci_data.values == -1]
ci_report.drop_duplicates(inplace=True)
# ------------------------------------------SCREENING REPORT- COURT INVOLVEMENT----------------------------------------------------------------------------------------
oci_data = oci_data = pd.read_csv('p_ScreeningExport_GetOtherCourtInvolvement.csv', parse_dates = ["ScreeningDate"])

# clean data to needed data types
oci_data['ScreeningDate'] = pd.to_datetime(oci_data['ScreeningDate'])

# fill in blank rows with -1
oci_data = oci_data.fillna(-1)

# delete uneeded rows  
oci_data.drop(["ScreeningID","LastName","FirstName", "MiddleName","DOB", "OutstandingWarrantStatus", "ProbationStatus", "PendingCriminalOffenses", "PriorAdjudications", "PreConvictionNonDomestic",
"PreConvictionDomestic","PreCourtFailToAppear", "PreDrugCourtParticipation","PreDrugCourtParticipationOther"],axis=1,inplace= True)

# choose date range for cohort
oci_data =oci_data[(oci_data['ScreeningDate']>= start) & (oci_data['ScreeningDate']<= end)]  

# find rows with blank and remove duplicate rows
oci_report = oci_data[oci_data.values == -1]
oci_report.drop_duplicates(inplace=True)



# ------------------------------------------SCREENING REPORT- SCREENING AND ASSESSMENT----------------------------------------------------------------------------------------
sa_data = sa_data = pd.read_csv('p_ScreeningExport_GetScreeningAssessments.csv', parse_dates = ["ScreeningDate"])

# clean data to needed data types
sa_data['ScreeningDate'] = pd.to_datetime(sa_data['ScreeningDate'])

# fill in blank rows with -1
sa_data = sa_data.fillna(-1)

# delete uneeded rows  
sa_data.drop(["ScreeningID","LastName","FirstName", "Gender", "Ethnicity", "MiddleName","DOB", "ScreeningCaseManager", "SecondaryDrugOfChoice", "TertiaryDrugOfChoice", "Comment",
"ViolenceRisk", "RecidivismRisk", "FailToAppearRisk", "CommunityNonCompRisk", "OverallRisk", "OverallRiskOther", "ViolenceRiskOther", "RecidivismRiskOther",
"FailToAppearRiskOther", "CommunityNonCompRiskOther", "CriminalRecidivismRisk", "FamilyParenting", "EducationEmployement" ,"PeerRelations", "SubstanceAbuse", "LeisureRecreation",
"PersonalityBehavior", "AttitudesOrientation", "AssessmentType" , "ToolAdministeredOther", "Score", "PrimaryDiagnosis", "SecondaryDiagnosis", "IVDrugUser", "TimingOfAdministration"],axis=1,inplace= True)

# choose date range for cohort
sa_data =sa_data[(sa_data['ScreeningDate']>= start) & (sa_data['ScreeningDate']<= end)]  

# find rows with blank and remove duplicate rows
sa_report = sa_data[sa_data.values == -1]
sa_report.drop_duplicates(inplace=True)

# ------------------------------------------SCREENING REPORT- SCREENING AND ASSESSMENT----------------------------------------------------------------------------------------

dx_data = dx_data = pd.read_csv('p_ScreeningExport_GetDiagnosis.csv', parse_dates = ["ScreeningDate"])

# clean data to needed data types
dx_data['ScreeningDate'] = pd.to_datetime(dx_data['ScreeningDate'])

# fill in blank rows with -1
dx_data = dx_data.fillna(-1)

# delete uneeded rows  
dx_data.drop(["ScreeningID","LastName","FirstName", "MiddleName","DOB", "SecondaryDiagnosis","OtherDiagnosis1", "OtherDiagnosis2", "OtherDiagnosis3"],axis=1,inplace= True)

# choose date range for cohort
dx_data = dx_data[(dx_data['ScreeningDate']>= start) & (dx_data['ScreeningDate']<= end)]  

# find rows with blank and remove duplicate rows
dx_report = dx_data[dx_data.values == -1]
dx_report.drop_duplicates(inplace=True)

# ------------------------------------------SCREENING REPORT- RELATED PARTIES---------------------------------------------------------------------------------------

rp_data  = pd.read_csv('p_ScreeningExport_GetRelatedParties.csv', parse_dates = ["ScreeningDate"])

# clean data to needed data types
rp_data['ScreeningDate'] = pd.to_datetime(rp_data['ScreeningDate'])

# fill in blank rows with -1
rp_data = rp_data.fillna(-1)

# delete uneeded rows  
rp_data.drop(["DOB","ScreeningID","LastName","FirstName", "MiddleName","RelatedPartyDOB","Gender", "Ethnicity","Address1", "City", "State", "InfantWithPrenatalSubstanceExposure",
 "LivingSituation", "ChildCurrentCustodyStatus", "CurrentChildSupport", "Issues", "Attorney"],axis=1,inplace= True)

# choose date range for cohort
rp_data =rp_data[(rp_data['ScreeningDate']>= start) & (rp_data['ScreeningDate']<= end)]  

# find rows with blank and remove duplicate rows
rp_report = rp_data[rp_data.values == -1]
rp_report.drop_duplicates(inplace=True)


# ------------------------------------------SCREENING REPORT- Veteran Info---------------------------------------------------------------------------------------

mi_data  = pd.read_csv('p_ScreeningExport_GetVeteranInfo.csv', parse_dates = ["ScreeningDate"])

# clean data to needed data types
mi_data['ScreeningDate'] = pd.to_datetime(mi_data['ScreeningDate'])

# fill in blank rows with -1
mi_data = mi_data.fillna(-1)

# delete uneeded rows  
mi_data.drop(["DOB","ScreeningID","LastName","FirstName", "MiddleName","Participant", "EnlistmentOrCommissioningDate", "MilitaryOccupationalSpecialty",
"DateofMilitaryDischarge", "MilitaryDischargeStatus", "DetailsOfDischarge", "MilitaryRank","HasServedInUSMilitary", "ConvictionsPriorToMilitaryService", "DDForm214OrOtherSource","ConvictionsPriorToMilitaryServiceHighestOffense",
"AwardsAndDecorations", "RankReduction", "MilitaryIncarceration", "DeployedAbroad", "CombatZone", "CombatZoneStationedOutcome", "WitnessorInvolvementNumberOfTimes", "PTSD", "PTSDDetails",
"TBI", "TBIDetails", "IED_HME", "IED_HME_Details", "MST","MSTDetails", "SentToVAVJOReferralDate", "VeteransAssociationOrGroupMembership","VeteransAssociationOrGroupMembershipDetails",
"ReceivingDisabilityCompensationFromtheVAPercentDisabled","UtilizingServicesFromTheVetCenter","ReceivingDisabilityCompensationFromtheVA", "UtilizingServicesFromTheVetCenterDetails",  "HasSuicideRisk"],axis=1,inplace= True)

# choose date range for cohort
mi_data =mi_data[(mi_data['ScreeningDate']>= start) & (mi_data['ScreeningDate']<= end)]  

# find rows with blank and remove duplicate rows
mi_report = mi_data[mi_data.values == -1]
mi_report.drop_duplicates(inplace=True)


# ------------------------------------------MILITARY REPORT---------------------------------------------------------------------------------------

mil_data  = pd.read_csv('MilitaryServiceReport.csv', parse_dates = ["AdmissionDate"])

# clean data to needed data types
mil_data['AdmissionDate'] = pd.to_datetime(mil_data['AdmissionDate'])

# fill in blank rows with -1
mil_data["MilitaryDischargeDate"] = mil_data["MilitaryDischargeDate"].replace([np.nan], "NA")
mil_data = mil_data.fillna(-1)
#####cross reference discharge date and reason######
mil_data['MilitaryDischargeReason']=mil_data['MilitaryDischargeReason'].mask(mil_data['MilitaryDischargeReason'].eq(-1),mil_data['MilitaryDischargeDate'])
mil_data.loc[mil_data['MilitaryDischargeReason'] != "NA", 'MilitaryDischargeDate'] = -1


# delete uneeded rows  
mil_data.drop(["CaseID","Clerk Case Number","ClientID","FullName", "Gender", "Race", "DOB", "CaseManager", "Judge", "CasePhase", "CaseStatus",  "MilitaryRank",
"Months Deployed Abroad", "Deployment Location", "Occupational Specialty", "Years of Service", "Suicide Risk", "Enlistment Date", "Conviction prior to Military", "Awards & Decorations",
"Discplinary Action/Rank Reduction", "Military Incarceration", "Number of Deployments abroad", "Conflict Eras of Service", "Number of Times Witness or Involvement", "Date Assessment Received from VA/VJO",
"Veterans Association Membership", "Disability Compensation from VA", "Vet Center Utilization", "Veteran Mentor Assigned", "Veteran Mentor Name", "Veteran Mentor Assigned Date"
],axis=1,inplace= True)

# choose date range for cohort
mil_data =mil_data[(mil_data['AdmissionDate']>= start) & (mil_data['AdmissionDate']<= end)]  

# find rows with blank and remove duplicate rows
mil_report = mil_data[mil_data.values == -1]
mil_report.drop_duplicates(inplace=True)

# ------------------------------------------SCREENING REPORT- Mental Health---------------------------------------------------------------------------------------
men_data  = pd.read_csv('p_ScreeningExport_GetMentalHealthHistory.csv', parse_dates = ["ScreeningDate"])

# clean data to needed data types
men_data['ScreeningDate'] = pd.to_datetime(men_data['ScreeningDate'])

# fill in blank rows with -1
men_data = men_data.fillna(-1)
men_data.loc[men_data['DefendantCompetent'] == "Y", 'CompetencyEvaluationOrderedDate'] = "NA"
men_data.loc[men_data['DefendantCompetent'] == "Y", 'CompetencyEvaluationReceivedDate'] = "NA"
men_data.loc[men_data['DefendantCompetent'] == "Y", 'AgeOfOnsetOfMentalHealthDisorderYears'] = "NA"


# delete uneeded rows  
men_data.drop(["DOB","ScreeningID","LastName","FirstName", "MiddleName","ReferralInformationSummary", "LevelOfCare", "RelatedMentalIllness"],axis=1,inplace= True)

# choose date range for cohort
men_data =men_data[(men_data['ScreeningDate']>= start) & (men_data['ScreeningDate']<= end)]  

# find rows with blank and remove duplicate rows
men_report = men_data[men_data.values == -1]
men_report.drop_duplicates(inplace=True)

# ------------------------------------------SCREENING REPORT-Family Information---------------------------------------------------------------------------------------
fam_data  = pd.read_csv('p_ScreeningExport_GetFamilyInfo.csv', parse_dates = ["ScreeningDate"])

# clean data to needed data types
fam_data['ScreeningDate'] = pd.to_datetime(fam_data['ScreeningDate'])

# fill in blank rows with -1
fam_data = fam_data.fillna(-1)

# delete uneeded rows  
fam_data.drop(["DOB","ScreeningID","LastName","FirstName", "MiddleName", "NumberOfDependantChildren","ChildCustodayOfAllChildrenAtAdmission",
"ChildVisitationRightsOfAllChildrenAtAdmission"],axis=1,inplace= True)

# choose date range for cohort
fam_data =fam_data[(fam_data['ScreeningDate']>= start) & (fam_data['ScreeningDate']<= end)]  

# find rows with blank and remove duplicate rows
fam_report = fam_data[fam_data.values == -1]
fam_report.drop_duplicates(inplace=True)

# ------------------------------------------PREGNANCY REPORT---------------------------------------------------------------------------------------
p_data  = pd.read_csv('PregnancyReport.csv', parse_dates = ["AdmissionDate"])

# clean data to needed data types
p_data['AdmissionDate'] = pd.to_datetime(p_data['AdmissionDate'])

# fill in blank rows with -1
p_data = p_data.fillna(-1)

p_data.loc[p_data.Pregnant == "No", 'Outcome'] = "NA"
p_data.loc[p_data.Pregnant == "No", 'DueDate'] = "NA"


#cl_data['Discharge Reason']=cl_data['Discharge Reason'].mask(cl_data['Discharge Reason'].eq(-1),cl_data['Discharge Date'])
# delete uneeded rows  
p_data.drop([ "CaseManager","Clerk Case Number" ,"FullName", "Judge", "Gender", "ChildDOB","CurrentMedicalConditions", "PharmacologicalIntervention", "PharmacologicalInterventionComment",
"CurrentMedications", "HistoryOfMedicalConditions", "MedicalHealthNotes"
],axis=1,inplace= True)

# choose date range for cohort
p_data =p_data[(p_data['AdmissionDate']>= start) & (p_data['AdmissionDate']<= end)]  

# find rows with blank and remove duplicate rows
p_report = p_data[p_data.values == -1]
p_report.drop_duplicates(inplace=True)

# ------------------------------------------TREATMENT ATTENDANCE REPORT---------------------------------------------------------------------------------------
tx_data  = pd.read_csv('TreatmentAttendanceReport.csv', parse_dates = ["Admit Date"])

# clean data to needed data types
tx_data['Admit Date'] = pd.to_datetime(tx_data['Admit Date'])

# fill in blank rows with -1
tx_data = tx_data.fillna(-1)
tx_data.loc[tx_data["Discharge Date"] == -1, 'Discharge Date'] = "NA"
tx_data.loc[tx_data["Discharge Date"] == "NA", 'Discharge Reason'] = "NA"


# delete uneeded rows  
tx_data.drop([ "Case Manager","Full Name", "Judge", "Race","DOB", "Gender", "Case Status","Service Units", "TreatmentFor", "Session(s)", "Missed Sessions"
],axis=1,inplace= True)

# choose date range for cohort
tx_data =tx_data[(tx_data['Admit Date']>= start) & (tx_data['Admit Date']<= end)]  

# find rows with blank and remove duplicate rows
tx_report = tx_data[tx_data.values == -1]
tx_report.drop_duplicates(inplace=True)

# ------------------------------------------ANCILLARY PROGRESS REPORT---------------------------------------------------------------------------------------
a_data  = pd.read_csv('AncillaryProgressReport.csv', parse_dates = ["AdmissionDate"])

# clean data to needed data types
a_data['AdmissionDate'] = pd.to_datetime(a_data['AdmissionDate'])
a_data.loc[a_data["Status"] == "Ongoing", 'DateEnded'] = "NA"
a_data.loc[a_data["Status"] == "In Progress", 'DateEnded'] = "NA"

# fill in blank rows with -1
a_data = a_data.fillna(-1)


# delete uneeded rows  
a_data.drop([ "CaseManager","FullName", "Judge", "Race","DOB", "Gender", "CaseStatus",  "ServiceUnits", "ContactHours", "ServiceFor","Sessions", "MissedSessions"
],axis=1,inplace= True)

# choose date range for cohort
a_data =a_data[(a_data['AdmissionDate']>= start) & (a_data['AdmissionDate']<= end)]  

# find rows with blank and remove duplicate rows
a_report = a_data[a_data.values == -1]
a_report.drop_duplicates(inplace=True)

# ------------------------------------------DRUG TESTING REPORT---------------------------------------------------------------------------------------
dr_data  = pd.read_csv('DrugTestingReport.csv')
# delete uneeded rows  
dr_data.drop([ "Clerk Case Number", "FullName", "Judge", "CaseManager"],axis=1,inplace= True)

dr_report = dr_data

# ------------------------------------------INCENTIVE REPORT---------------------------------------------------------------------------------------
in_data  = pd.read_csv('IncentiveReport.csv')


# delete uneeded rows  
in_data.drop(["Clerk Case Number", "AdmissionDate","FullName","CasePhase","Gender", "Notes", "Details", "DOB", "Judge", "Ethnicity-HispanicLatinoOrSpanishOrigin","CaseManager"],axis=1,inplace= True)

in_report = in_data
# ------------------------------------------Jail REPORT---------------------------------------------------------------------------------------
j_data  = pd.read_csv('JailReport.csv', parse_dates = ["AdmissionDate"])

# clean data to needed data types
j_data['AdmissionDate'] = pd.to_datetime(j_data['AdmissionDate'])
j_data.loc[j_data["CasePhase"] != "Discharged", 'JailReleasedDate'] = "NA"

# fill in blank rows with -1
j_data = j_data.fillna(-1)


# delete uneeded rows  
j_data.drop([ "Clerk Case Number", "FullName", "Gender", "Race", "DOB", "CaseManager", "Judge", "CaseStatus", "JailReason"
],axis=1,inplace= True)

# find rows with blank and remove duplicate rows
j_report = j_data[j_data.values == -1]
j_report.drop_duplicates(inplace=True)

# ------------------------------------------SUSPENSION REPORT---------------------------------------------------------------------------------------
sus_data  = pd.read_csv('SuspendedReport.csv')


# delete uneeded rows  "
sus_data.drop(["Clerk Case Number", "CaseStatus","DaysSuspended", "ActivatedDate","CaseManager","Race","AdmissionDate","FullName","CasePhase","Gender", "DischargeDate", "DischargeReason","DOB", "Judge"],axis=1,inplace= True)

sus_report = sus_data
# ------------------------------------------------------write data to report-----------------------------------------------------
writer = pd.ExcelWriter(report_name, engine= 'xlsxwriter')
cl_report.to_excel(writer,index=False, sheet_name= "caselist")
ci_report.to_excel(writer,index=False, sheet_name= "caseinformation")
oci_report.to_excel(writer,index=False, sheet_name = "othercourtinvolvement")
sa_report.to_excel(writer,index=False, sheet_name = "screeningassessment")
men_report.to_excel(writer,index=False, sheet_name = "mentalhealth")
dx_report.to_excel(writer,index=False, sheet_name = "diagnosis")
rp_report.to_excel(writer,index=False, sheet_name = "relatedparties")
fam_report.to_excel(writer,index=False, sheet_name = "familyinformation")
mi_report.to_excel(writer,index=False, sheet_name = "screening-military")
mil_report.to_excel(writer,index=False, sheet_name = "military")
p_report.to_excel(writer,index=False, sheet_name = "pregnancy")
tx_report.to_excel(writer,index=False, sheet_name = "treatmentattendance")
a_report.to_excel(writer,index=False, sheet_name = "ancillaryprogress")
dr_report.to_excel(writer,index=False, sheet_name = "drugtests")
in_report.to_excel(writer,index=False, sheet_name = "incentives")
j_report.to_excel(writer,index=False, sheet_name = "jailreport")
sus_report.to_excel(writer,index=False, sheet_name = "suspensionreport")
workbook = writer.book
redfill = workbook.add_format({'bg_color': '#FFC7CE'})
worksheet = writer.sheets['caselist']
worksheet.conditional_format('A1:EP15000',{
    'type':     'cell',
    'criteria': '==',
    'value':    -1,
    'format':   redfill})
worksheet1 = writer.sheets['caseinformation']
worksheet1.conditional_format('A1:EP15000',{
    'type':     'cell',
    'criteria': '==',
    'value':    -1,
    'format':   redfill})
worksheet2 = writer.sheets["othercourtinvolvement"]
worksheet2.conditional_format('A1:EP15000',{
    'type':     'cell',
    'criteria': '==',
    'value':    -1,
    'format':   redfill})
worksheet3 = writer.sheets["screeningassessment"]
worksheet3.conditional_format('A1:EP15000',{
    'type':     'cell',
    'criteria': '==',
    'value':    -1,
    'format':   redfill})
worksheet15 = writer.sheets["mentalhealth"]
worksheet15.conditional_format('A1:EP15000',{
    'type':     'cell',
    'criteria': '==',
    'value':    -1,
    'format':   redfill})
worksheet4 = writer.sheets["diagnosis"]
worksheet4.conditional_format('A1:EP15000',{
    'type':     'cell',
    'criteria': '==',
    'value':    -1,
    'format':   redfill})
worksheet5 = writer.sheets["relatedparties"]
worksheet5.conditional_format('A1:EP15000',{
    'type':     'cell',
    'criteria': '==',
    'value':    -1,
    'format':   redfill})
worksheet6 = writer.sheets["screening-military"]
worksheet6.conditional_format('A1:EP15000',{
    'type':     'cell',
    'criteria': '==',
    'value':    -1,
    'format':   redfill})
worksheet7 = writer.sheets["military"]
worksheet7.conditional_format('A1:EP15000',{
    'type':     'cell',
    'criteria': '==',
    'value':    -1,
    'format':   redfill})
worksheet8 = writer.sheets["pregnancy"]
worksheet8.conditional_format('A1:EP15000',{
    'type':     'cell',
    'criteria': '==',
    'value':    -1,
    'format':   redfill})
worksheet9 = writer.sheets["treatmentattendance"]
worksheet9.conditional_format('A1:EP15000',{
    'type':     'cell',
    'criteria': '==',
    'value':    -1,
    'format':   redfill})
worksheet10 = writer.sheets["ancillaryprogress"]
worksheet10.conditional_format('A1:EP15000',{
    'type':     'cell',
    'criteria': '==',
    'value':    -1,
    'format':   redfill})
worksheet13 = writer.sheets["jailreport"]
worksheet13.conditional_format('A1:EP15000',{
    'type':     'cell',
    'criteria': '==',
    'value':    -1,
    'format':   redfill})
worksheet16 = writer.sheets["familyinformation"]
worksheet16.conditional_format('A1:EP15000',{
    'type':     'cell',
    'criteria': '==',
    'value':    -1,
    'format':   redfill})

writer.save()
writer.close()
    




SyntaxError: invalid syntax (Temp/ipykernel_3420/647647742.py, line 43)